## Deep Learning

In [1]:
import os
import pandas as pd
import math
import seaborn as sns
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import History
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import History
from keras.callbacks import EarlyStopping, ModelCheckpoint
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns  #Seaborn is another powerful visulization library for Python
from sklearn.linear_model import Ridge
import os

C:\Users\jmatney\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [14]:
path = "C:\\Users\jmatney\Documents\GitHub\CRRA\data\hybrid"
df = pd.read_csv(os.path.join(path, "CRRA_data.csv"))

In [15]:
df.head(5)

,x,y,Ind_DEM_m,Ind_DEM_Std_ft,Ind_VOFD_ft,Ind_HOFD_ft,Ind_Point_Drainage_Area_sqmi,Ind_Catchment_Drainage_Area_sqmi,Ind_Slope_deg,Ind_Wetness_index,...,Dep_500yr_Depth_ft,Other_Catchment_ID,Other_HasBLE,Other_Soils_Mukey,Ind_Soil_Drainage_Class_Typical_index,Ind_Soil_Drainage_Class_Wettest_index,Ind_Soil_Type_index,Ind_Soil_Flood_Freq_Typical_index,Ind_Soil_Flood_Freq_Wettest_index,Ind_Soil_Hydric_Soils_percent
0,224898.134159,4.715585e+06,558.8,0.219963,82.899660,2412.499300,0.000000,1.110116,1.012812,8.640562,...,0.000000,250,1,295347,6,6,7,5,5,95
1,225928.134159,4.719415e+06,485.8,1.815682,0.099976,72.426410,0.000000,14.192085,3.754432,7.329027,...,0.386475,101,1,295357,7,7,4,5,5,95
2,226568.134159,4.725335e+06,508.7,0.369994,0.200012,24.142136,0.000309,2.846564,1.406812,10.509094,...,0.000000,7,1,295438,7,7,4,2,2,95
3,228068.134159,4.723095e+06,537.7,1.561457,5.500000,264.852780,0.000000,0.802780,6.989694,6.703982,...,0.000000,21,1,295307,3,3,5,5,5,0
4,219088.134159,4.719115e+06,545.2,0.360797,73.099550,4091.616500,0.000000,0.763166,1.505312,8.244166,...,0.000000,160,1,295338,5,5,7,5,5,5


In [21]:
# df_data = df_data.loc[:, df_data.columns != 'x']

In [22]:
df_data.columns

Index(['Ind_DEM_m', 'Ind_DEM_Std_ft', 'Ind_VOFD_ft', 'Ind_HOFD_ft',
       'Ind_Point_Drainage_Area_sqmi', 'Ind_Catchment_Drainage_Area_sqmi',
       'Ind_Slope_deg', 'Ind_Wetness_index', 'Ind_Stream_Lakes_boolean',
       'Ind_NYS_Stream_boolean', 'Ind_Stream_Order_index',
       'Ind_Land_Use_class', 'Ind_Annual_Rainfall_mm', 'Dep_100yr_Depth_ft',
       'Dep_100yr_10pct_Depth_ft', 'Dep_100yr_20pct_Depth_ft',
       'Dep_100yr_plus_Depth_ft', 'Dep_500yr_Depth_ft', 'Other_Catchment_ID',
       'Other_HasBLE', 'Other_Soils_Mukey',
       'Ind_Soil_Drainage_Class_Typical_index',
       'Ind_Soil_Drainage_Class_Wettest_index', 'Ind_Soil_Type_index',
       'Ind_Soil_Flood_Freq_Typical_index',
       'Ind_Soil_Flood_Freq_Wettest_index', 'Ind_Soil_Hydric_Soils_percent'],
      dtype='object')

In [35]:
#Define a variable for each type of feature
target = ["Dep_100yr_Depth_ft"]
columns = [x for x in df if x != "Dep_100yr_Depth_ft"]
numeric_columns = [x for x in columns if x != "x"]
numeric_columns = [x for x in numeric_columns if x != "y"]

In [25]:
# Normalise
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df_data)

In [26]:
from keras.models import Model
from keras.layers import Input, Dense
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
# Fixed dimensions
input_dim = df_data.shape[1]  # 8
encoding_dim = 3
# Number of neurons in each Layer [8, 6, 4, 3, ...] of encoders
input_layer = Input(shape=(input_dim, ))
encoder_layer_1 = Dense(6, activation="tanh", activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder_layer_2 = Dense(4, activation="tanh")(encoder_layer_1)
encoder_layer_3 = Dense(encoding_dim, activation="tanh")(encoder_layer_2)

In [27]:
# Crear encoder model
encoder = Model(inputs=input_layer, outputs=encoder_layer_3)
# Use the model to predict the factors which sum up the information of interest rates.
encoded_data = pd.DataFrame(encoder.predict(data_scaled))
encoded_data.columns = ['factor_1', 'factor_2', 'factor_3']

In [28]:
encoded_data

,factor_1,factor_2,factor_3
0,-0.217496,-0.645923,0.174423
1,0.229068,-0.179289,-0.134379
2,-0.136696,-0.317205,0.085138
3,0.288527,-0.259364,-0.169756
4,0.311689,-0.105847,-0.182365
...,...,...,...
5460088,0.145237,-0.404794,-0.017405
5460089,0.494128,0.198012,-0.285320
5460090,0.264255,-0.248808,-0.085705
5460091,0.227254,-0.243087,-0.105533


## Min Max Scaler

In [30]:
X = df_data.loc[:, df_data.columns != 'Dep_100yr_Depth_ft']
y = df_data.loc[:, df_data.columns == 'Dep_100yr_Depth_ft']

In [31]:
scaler = MinMaxScaler()

# create target scaler object
scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
scalarX.fit(X)
scalarY.fit(y)

MinMaxScaler()

## Train Test Split

In [36]:
#Create train and test dataset with an 80:20 split
x_train, x_test, y_train, y_test = train_test_split(df_data[numeric_columns],df_data[target],test_size=0.2,random_state=2018)
# Further divide training dataset into train and validation dataset with an 90:10 split
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.1,random_state=2018)

# transform target variables
y_train = scalarY.transform(y_train)
y_test = scalarY.transform(y_test)
y_val = scalarY.transform(y_val)

x_train = scalarX.transform(x_train)
x_test = scalarX.transform(x_test)
x_val = scalarX.transform(x_val)

#Check the sizes of all newly created datasets
print("Shape of x_train:",x_train.shape)
print("Shape of x_val:",x_val.shape)
print("Shape of x_test:",x_test.shape)
print("Shape of y_train:",y_train.shape)
print("Shape of y_val:",y_val.shape)
print("Shape of y_test:",y_test.shape)

Shape of x_train: (3931266, 26)
Shape of x_val: (436808, 26)
Shape of x_test: (1092019, 26)
Shape of y_train: (3931266, 1)
Shape of y_val: (436808, 1)
Shape of y_test: (1092019, 1)


In [ ]:

# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_loss', patience=4)]

model4 = Sequential()
model4.add(Dense(350,input_dim = 26,activation="relu"))
model4.add(Dropout(0.2, input_shape=(26,)))
model4.add(Dense(350,activation="relu"))
model4.add(Dropout(0.2, input_shape=(26,)))
model4.add(Dense(350,activation="relu"))
model4.add(Dropout(0.2, input_shape=(26,)))
model4.add(Dense(350,activation="relu"))
model4.add(Dropout(0.2, input_shape=(26,)))
model4.add(Dense(350,activation="relu"))
model4.add(Dropout(0.2, input_shape=(26,)))
model4.add(Dense(1,activation = "linear"))

model4.compile(optimizer='adam',loss="mean_squared_error",
               metrics=[tf.keras.metrics.RootMeanSquaredError()])

model4.fit(x_train,y_train, 
           validation_data=(x_val,y_val), 
           epochs=10,
           batch_size=64)
#,callbacks=callbacks)

result = model4.evaluate(x_test,y_test)

for i in range(len(model4.metrics_names)):
    print("Metric ",model4.metrics_names[i],":",str(round(result[i],2)))
    


Train on 3931266 samples, validate on 436808 samples
Epoch 1/10
 580800/3931266 [===>..........................] - ETA: 9:59 - loss: 0.0012 - root_mean_squared_error: 0.0547